In [ ]:
#解压数据
!unzip -o data/data86770/seg.zip -d /home/aistudio/work

In [ ]:
#已经克隆了，不用再克隆
# !git clone https://gitee.com/PaddlePaddle/PaddleSeg

Cloning into 'PaddleSeg'...
remote: Enumerating objects: 11141, done.
remote: Counting objects: 100% (11141/11141), done.
remote: Compressing objects: 100% (5537/5537), done.
remote: Total 11141 (delta 7556), reused 8285 (delta 5439), pack-reused 0
Receiving objects: 100% (11141/11141), 157.00 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (7556/7556), done.
Checking connectivity... done.


### 生成train.txt 和val.txt

In [ ]:
import random
import os
random.seed(2020)
mask_dir  = '/home/aistudio/work/seg/Train/masks'
img_dir = '/home/aistudio/work/seg/Train/fundus_image'
path_list = list()
for img in os.listdir(img_dir):
    img_path = os.path.join(img_dir,img)
    mask_path = os.path.join(mask_dir,img.replace('jpg', 'png'))
    path_list.append((img_path, mask_path))
random.shuffle(path_list)
ratio = 0.8
train_f = open('/home/aistudio/work/seg/Train/train.txt','w') 
val_f = open('/home/aistudio/work/seg/Train/val.txt' ,'w')

for i ,content in enumerate(path_list):
    img, mask = content
    text = img + ' ' + mask + '\n'
    if i < len(path_list) * ratio:
        train_f.write(text)
    else:
        val_f.write(text)
train_f.close()
val_f.close()

### 配置文件如下
```
batch_size: 4
iters: 16000

train_dataset:
  type: Dataset
  dataset_root: /home/aistudio/work/seg/Train/
  train_path: /home/aistudio/work/seg/Train/train.txt
  num_classes: 2
  transforms:
    - type: Resize
      target_size: [512, 512]
    # - type: RandomRotation
    #   max_rotation: 15
    - type: RandomHorizontalFlip
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: /home/aistudio/work/seg/Train/
  val_path: /home/aistudio/work/seg/Train/val.txt
  num_classes: 2
  transforms:
    - type: Resize
      target_size: [512, 512]
    - type: Normalize
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

learning_rate:
  value: 0.00125
  decay:
    type: poly
    power: 0.9
    end_lr: 0.0

loss:
  types:
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: DiceLoss
      coef: [0.7, 0.3]
  coef: [1]

model:
  type: UNet
  num_classes: 2
  use_deconv: False
  pretrained: /home/aistudio/unetmodel.pdparams

```

### 开始训练

In [ ]:

%cd /home/aistudio/PaddleSeg
!python train.py \
       --config configs/unet_PALM.yml \
       --do_eval \
       --use_vdl \
       --save_interval 480 \
       --save_dir output_unet_PALMoutput

/home/aistudio/PaddleSeg
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-05-06 21:35:50 [INFO]	
------------Environment Information-------------
platform: Linux-4.13.0-36-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICE

### 验证

In [ ]:
!python val.py --config configs/unet_PALM.yml  --model_path output_unet_PALMoutput/best_model/model.pdparams 

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-05-07 18:16:17 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 16000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    type: poly
  value: 0.00125
loss:
  coef:
  - 1
  types:
  - coef:
    - 0.7
    - 0.3
    losses:
    - type: CrossEntropyLoss
    - type: DiceLoss
    type: MixedLoss
model:
  num_classes: 2

### 预测

In [ ]:
!python predict.py \
       --config configs/unet_PALM.yml \
       --model_path output_unet_PALMoutput/best_model/model.pdparams \
       --image_path /home/aistudio/work/seg/test \
       --save_dir output_unet_PALMoutput/result

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-05-07 18:22:39 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 16000
learning_rate:
  decay:
    end_lr: 0.0
    power: 0.9
    type: poly
  value: 0.00125
loss:
  coef:
  - 1
  types:
  - coef:
    - 0.7
    - 0.3
    losses:
    - type: CrossEntropyLoss
    - type: DiceLoss
    type: MixedLoss
model:
  num_classes: 2

### 生成结果

In [ ]:
import os 
import cv2
result_path = '/home/aistudio/PaddleSeg/output_unet_PALMoutput/result/pseudo_color_prediction'
dist_path = '/home/aistudio/Disc_Segmentation'
for img_name in os.listdir(result_path):
    img_path = os.path.join(result_path, img_name)
    img = cv2.imread(img_path)
    g  = img[:,:,1]
    ret, result = cv2.threshold(g, 127,255, cv2.THRESH_BINARY_INV)
    cv2.imwrite(os.path.join(dist_path,img_name), result)


### 假如预测中出现多个不连通的区域，只保留最大的区域

In [ ]:
import os 
import cv2
import matplotlib.pyplot as plt
def cnt_area(cnt):
    area = cv2.contourArea(cnt)
    return area

result_path = '/home/aistudio/PaddleSeg/output_unet_PALMoutput/result/pseudo_color_prediction'
dist_path = '/home/aistudio/Disc_Segmentation'
for img_name in os.listdir(result_path):
    img_path = os.path.join(result_path, img_name)
    img = cv2.imread(img_path)
    g  = img[:,:,1]
    ret, threshold = cv2.threshold(g, 127,255, cv2.THRESH_BINARY)


    contours, hierarch = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours.sort(key=cnt_area, reverse=True)
    if len(contours) > 1:
        for i in range(1,len(contours)):
            cv2.drawContours(threshold, [contours[i]], 0, 0, -1)
    _,result = cv2.threshold(threshold, 127, 255, cv2.THRESH_BINARY_INV)
    cv2.imwrite(os.path.join(dist_path, img_name), result)


### 总结
模型构建思路及调优过程（可具体包括：思路框架图、思路步骤详述、模型应用+调优过程）

【模型】Unet

【数据增强】图片大小512x512，水平翻转，对比度随机改变等数据增强

【对预测结果进行处理】假如预测中出现多个不连通的区域，只保留最大的区域